In [ ]:
import pygui as pgui
pgui.create_interface()


# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3

Output()

Reloading Decks
Loading Decks with arguments ['--username', 'tmind', '--mode', 'update']
Requesting Data from Website: tmind,deck,


Saving Decks: 100%|██████████| 185/185 [00:02<00:00, 73.24it/s]


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: tmind.Card index: _id_ dup key: { _id: "s1aa1aetherforge-oracle" }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 's1aa1aetherforge-oracle'}, 'op': {'name': 'Aetherforge Oracle', 'title': 'Aetherforge Oracle', 'faction': '', 'cardType': 'Creature', 'cardSubType': 'Mage', 'betrayer': False, 'sortValue': '', 'crossFaction': '', 'cardSetId': '', '_id': 's1aa1aetherforge-oracle', 'rarity': 'Common', 'levels': {'1': {'health': 5, 'attack': 3, 'text': '<b>Deploy:</b> <b>Upgrade</b> a card in your hand, then discard it.'}, '2': {'health': 10, 'attack': 6, 'text': '<b>Deploy:</b> <b>Upgrade</b> a card in your hand, then discard it.'}, '3': {'health': 13, 'attack': 9, 'text': '<b>Deploy:</b> <b>Upgrade</b> a card in your hand. \n \n When you <b>Upgrade</b> a card, you may <b>Upgrade</b> it up to two times instead.'}}, 'attack': {}, 'health': {}, '

In [11]:
import pygui as pgui
import pandas as pd
import qgrid
from IPython.display import display
from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables
from icecream import ic

df = pgui.collection_to_df('Deck')

GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)
db_name = myDB.get_current_db_name()

# Get all Decks from the database
deck_cursor = myDB.find('Deck', {})
df_decks = pd.DataFrame(list(deck_cursor))

# Create a DataFrame from the 'stats' sub-dictionary
df_decks_stats = pd.DataFrame()
for deck in myDB.find('Deck', {}):
    if 'stats' in deck:


        stats = deck.get('stats', {})
        # Convert the 'card_types' sub-dictionary into a DataFrame
        card_types_df = pd.DataFrame.from_dict(stats['card_types'], orient='index')
        creature_averages_df = pd.DataFrame.from_dict(stats['creature_averages'], orient='index')
           
        # Flatten the 'creature_averages' sub-dictionaries into a single-row DataFrame
        # Convert the dictionary into a DataFrame
        attack_dict = stats['creature_averages']['attack']
        attack_df = pd.DataFrame([attack_dict], columns=attack_dict.keys())
        attack_df.columns = ['A1', 'A2', 'A3']

        defense_dict = stats['creature_averages']['health']
        defense_df = pd.DataFrame([defense_dict], columns=defense_dict.keys())
        defense_df.columns = ['H1', 'H2', 'H3']

        # Combine the attack and defense DataFrames into a single DataFrame
        deck_stats_df = pd.concat([attack_df, defense_df], axis=1)
        deck_stats_df['name'] = deck['name']

        # Round each value in the DataFrame
        deck_stats_df = deck_stats_df.round(2)
                        
        # Extend record in df_decks with the deck_stats_df 
        df_decks_filtered = df_decks[[ 'registeredDate', 'name', 'cardSetName', 'faction', 'forgebornId']]
        
        # Set the common column as the index in both dataframes
        deck_stats_df.set_index('name', inplace=True)
        df_decks_filtered.set_index('name', inplace=True)

        # Merge the deck_stats_df with the deck_df_filtered
        merged_df = pd.concat([df_decks_filtered, deck_stats_df], axis=1)        
        display(merged_df)
        
        

qgrid_df_DeckStats = qgrid.show_grid(df_decks_stats, show_toolbar=True)            
display(qgrid_df_DeckStats)


# Merge the filtered dataframe with the deck stats dataframe
merged_df = df_filtered.merge(df_decks_stats, left_index=True, right_index=True)
qgrid_df = qgrid.show_grid(df_filtered, show_toolbar=True)            
display(qgrid_df)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/Users/tmind/Documents/Code/GitLab/SolDB/myenv/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2070, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tmind/Documents/Code/GitLab/SolDB/myenv/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2106, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# Create dataframe from Deck 
deck_df = pd.DataFrame({k: pd.Series(v) for k, v in deck.items()})